# The ABC of ATB (Automated Trading Backtest)

## A. Obtaining Data

In this part, we first retrieve data and save them as CSV files.We will first start with the easiest way to obtain data from Yahoo finance, and then look at other data sources in the future.

### A1. Getting Data from Google or  Yahoo Finance

In [3]:
# getting_data.py
"""
This code simply gets data from Google or Yahoo Finance
Input: symbols (list), start and end time
Output: CSV files that contain the time series of a stock for each symbol in the list
"""

# For compatibility with Py2 and Py3
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

# Import necessary functions
import datetime
import pandas as pd
import pandas_datareader.data as web
import sys

# Check Python and Pandas version
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)

# Inputs
today = datetime.date.today()
symbols = ['SPY', 'MAR', 'AMZN']
site = 'google'  # Options: google, yahoo, etc.
start = datetime.datetime(2010, 1, 1)
end = today  # Or we use "end = datetime.datetime(2017, 8, 17)" for example

if __name__ == "__main__":
    for s in symbols:
        # Getting the data
        data = web.DataReader(s, site, start, end)
        # Set the file name
        filename = s + ".csv"
        # Write to CSV file
        pd.DataFrame.to_csv(data, filename)
    print('Done.')


Python version 3.5.3 |Anaconda 4.0.0 (64-bit)| (default, Feb 22 2017, 21:28:42) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.20.3
Done.


# B. Event-Driven Trading Engine Implementation

In general, an event-driven software continually checks for new events and then performs actions based on these
events. To apply an event-driven approach to a backtesting system it is necessary to define our components (or objects) that will handle specific tasks:
    1. Event
    2. Event Queue
    3. DataHandler
    4. Strategy
    5. Portfolio
    6. ExecutionHandler
    7. Backtest 
 We will explain them in the following. 

## B1. Event

### Event

The Event calss is the base class of the system, which intuitively is what happening in the system. In particular, we consier four types of event, Market, Signal, Order and Fill. First, we define the general Event class.

In [4]:
# event.py
#%%
# Define all types of events.


class Event(object):
    """
    Event is base class providing an interface for all subsequent
    (inherited) events, that will trigger further events in the
    trading infrastructure.
    """
    pass

### Market Event

The MarketEvents are triggered when the outer while loop of the backtesting system starts over. It occurs when the DataHandler object receives a new update of market data for any symbols which are currently being tracked. It is used to trigger the Strategy object generating new trading signals. The event object simply contains an identification that it is a market event, with no other structure.

In [5]:
# event.py (cont.)

class MarketEvent(Event):
    """
    Handles the event of receiving a new market update with 
    corresponding bars.
    """

    def __init__(self):
        """
        Initialises the MarketEvent.
        """
        self.type = 'MARKET'


### Signal Event